In [2]:
import torch
import numpy as np

### Pytorch Basic operations

In [3]:
number = torch.tensor(4.)
number

tensor(4.)

In [4]:
number.dtype

torch.float32

In [5]:
t2 = torch.tensor([1., 2, 3, 4])
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

##### Gradient

We can combine tensors with the usual arithmetic operations. Using *requires_grad* in the initializaiton of a pytorch variabl, we signal to trace its gradient. 

In [6]:
# Create tensors.
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(5., requires_grad=True)
x, w, b

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

$$ y = w*x + b, \qquad x = 3, w = 4, b = 5 $$

In [7]:
# Arithmetic operations
y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

To compute derivates, we can invoke the *.backward* method

In [8]:
# Compute derivatives
y.backward()

In [9]:
# Display gradients
print('dy/dx:', x.grad)
print('dy/dw:', w.grad)
print('dy/db:', b.grad)

dy/dx: None
dy/dw: tensor(3.)
dy/db: tensor(1.)


#### Creation of tensors

In [11]:
t1 = torch.tensor([[1, 2], [3, 4], [5, 6]])
t2 = torch.full((3, 2), 42)
t2

tensor([[42, 42],
        [42, 42],
        [42, 42]])

In [14]:
t3 = torch.cat((t1, t2))
t3

tensor([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [42, 42],
        [42, 42],
        [42, 42]])

In [15]:
torch.sin(t3)

tensor([[ 0.8415,  0.9093],
        [ 0.1411, -0.7568],
        [-0.9589, -0.2794],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165],
        [-0.9165, -0.9165]])

In [18]:
x = np.random.rand(12).reshape(3, 4)

In [20]:
t4 = torch.from_numpy(x)
t4

tensor([[0.6271, 0.3833, 0.1654, 0.6483],
        [0.3568, 0.7267, 0.3471, 0.9081],
        [0.7958, 0.8098, 0.4222, 0.9586]], dtype=torch.float64)

# Linear regression


<img src="https://i.imgur.com/6Ujttb4.png">


In [21]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [22]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We start with random weights

In [25]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(f"Starting random weights:\n {w}\n")
print(f"Starting random bias:\n {b}")

Starting random weights:
 tensor([[ 0.8600,  1.2876, -1.8261],
        [-1.2874, -0.3288, -0.1104]], requires_grad=True)

Starting random bias:
 tensor([0.2218, 0.4882], requires_grad=True)


Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).

<img src="https://i.imgur.com/WGXLFvA.png">

In [27]:
def model(x, w, b):
    return x @ w.t() + b

In [28]:
# Generate predictions
preds = model(inputs, w, b)
print(preds)

tensor([[  70.7507, -120.2634],
        [  74.9226, -152.6581],
        [ 141.6676, -161.9712],
        [  75.7466, -149.0433],
        [  55.3458, -127.6289]], grad_fn=<AddBackward0>)


This results represent the number of Apples (col 1) and Oranges (col 2) for each region (row)

In [29]:
# Compare with targets
print(targets-preds)

tensor([[-14.7507, 190.2634],
        [  6.0774, 253.6581],
        [-22.6676, 294.9713],
        [-53.7466, 186.0433],
        [ 47.6542, 246.6289]], grad_fn=<SubBackward0>)


To evaluate how good/bad the model is doing we introduce a *loss* function

In [30]:
# MSE loss - average of squared residuals
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [31]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(28891.6523, grad_fn=<DivBackward0>)


In average each prediction differs from the actual value for a number of units equal to the square root of this *loss*. Our goal is to minimize the loss function uswith gradient

In [32]:
# Compute gradients
loss.backward()

In [33]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.8600,  1.2876, -1.8261],
        [-1.2874, -0.3288, -0.1104]], requires_grad=True)
tensor([[   937.9697,    245.4474,     42.5748],
        [-19725.6875, -21254.3906, -13134.2812]])


To decrease the loss we must make a small step in the direction opposite of the gradient

In [34]:
lr = 1e-5
with torch.no_grad():
    w -= w.grad * lr
    b -= b.grad * lr

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [35]:
# Let's verify that the loss is actually lower
loss = mse(model(inputs, w, b), targets)
print(loss)

tensor(19656.0898, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [36]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [42]:
# Train for 100 epochs
def train_model(w, b, k):
    loss_list = []
    lr = 1e-5
    for i in range(100):
        preds = model(inputs, w, b)
        loss = mse(preds, targets)
        loss_list.append(loss)
        print(f"epoch[{i}]: loss1 = {loss:.2f}")
        loss.backward()
        with torch.no_grad():
            w -= w.grad * lr
            b -= b.grad * lr
            if( i%k == 0):
                w.grad.zero_()
                b.grad.zero_()
    return loss_list

In [43]:
losses1 = train_model(w, b,10)

epoch[0]: loss1 = 124.91
epoch[1]: loss1 = 122.53
epoch[2]: loss1 = 122.30
epoch[3]: loss1 = 121.84
epoch[4]: loss1 = 121.16
epoch[5]: loss1 = 120.26
epoch[6]: loss1 = 119.13
epoch[7]: loss1 = 117.79
epoch[8]: loss1 = 116.24
epoch[9]: loss1 = 114.48
epoch[10]: loss1 = 112.53
epoch[11]: loss1 = 110.39
epoch[12]: loss1 = 110.18
epoch[13]: loss1 = 109.77
epoch[14]: loss1 = 109.16
epoch[15]: loss1 = 108.34
epoch[16]: loss1 = 107.33
epoch[17]: loss1 = 106.13
epoch[18]: loss1 = 104.73
epoch[19]: loss1 = 103.15
epoch[20]: loss1 = 101.40
epoch[21]: loss1 = 99.47
epoch[22]: loss1 = 99.28
epoch[23]: loss1 = 98.92
epoch[24]: loss1 = 98.36
epoch[25]: loss1 = 97.63
epoch[26]: loss1 = 96.72
epoch[27]: loss1 = 95.64
epoch[28]: loss1 = 94.38
epoch[29]: loss1 = 92.96
epoch[30]: loss1 = 91.38
epoch[31]: loss1 = 89.64
epoch[32]: loss1 = 89.48
epoch[33]: loss1 = 89.15
epoch[34]: loss1 = 88.65
epoch[35]: loss1 = 87.99
epoch[36]: loss1 = 87.17
epoch[37]: loss1 = 86.19
epoch[38]: loss1 = 85.06
epoch[39]: los

In [39]:
w2 = torch.randn(2, 3, requires_grad=True)
b2 = torch.randn(2, requires_grad=True)
losses2 = train_model(w2, b2, 1)

In [40]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [41]:
plt.plot(range(100), losses1, label="1")
plt.plot(range(100), losses2, label="2")
plt.legend()
plt.show()

NameError: name 'plt' is not defined

### Linear regression using PyTorch built-ins
We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

###### Data loading

In [917]:


# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [918]:
from torch.utils.data import TensorDataset

In [919]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [920]:
from torch.utils.data import DataLoader
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [921]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[103.,  43.,  36.],
        [ 68.,  97.,  70.],
        [102.,  43.,  37.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.]])
tensor([[ 20.,  38.],
        [102., 120.],
        [ 22.,  37.],
        [118., 132.],
        [ 21.,  38.]])


###### Model loading
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [922]:
import torch.nn as nn

# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3117,  0.1121,  0.0023],
        [-0.1589, -0.5105,  0.1858]], requires_grad=True)
Parameter containing:
tensor([0.3957, 0.3833], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [923]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.3117,  0.1121,  0.0023],
         [-0.1589, -0.5105,  0.1858]], requires_grad=True),
 Parameter containing:
 tensor([0.3957, 0.3833], requires_grad=True)]

In [924]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-14.7476, -37.4269],
        [-17.9550, -47.1046],
        [-11.5644, -71.0671],
        [-26.4928, -30.8973],
        [-10.1859, -46.5784],
        [-15.1715, -37.0753],
        [-18.0648, -46.4083],
        [-11.8738, -71.0401],
        [-26.0690, -31.2489],
        [ -9.8718, -46.2337],
        [-14.8574, -36.7306],
        [-18.3788, -46.7530],
        [-11.4546, -71.7634],
        [-26.8069, -31.2420],
        [ -9.7620, -46.9300]], grad_fn=<AddmmBackward>)

###### Loss function

In [925]:
# Import nn.functional
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

loss = loss_fn(model(inputs), targets)
print(loss)

tensor(15389.6973, grad_fn=<MseLossBackward>)


###### Optimizer

In [926]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. In complex models different branches of the structure can use different optimizers, so not all the parameters of the network must be passed.
Also, we can specify a learning rate that controls the amount by which the parameters are modified.

###### Training loop

In [927]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [928]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 358.9918
Epoch [20/100], Loss: 512.6168
Epoch [30/100], Loss: 55.0282
Epoch [40/100], Loss: 100.0373
Epoch [50/100], Loss: 119.1583
Epoch [60/100], Loss: 5.1483
Epoch [70/100], Loss: 25.4652
Epoch [80/100], Loss: 40.4350
Epoch [90/100], Loss: 19.7466
Epoch [100/100], Loss: 15.7992
